In [1]:
import sys
sys.path.append('..')
from model import Model, UnetModel
from process_data import preprocess, postprocess
import torch
from PIL import Image
import numpy as np

In [2]:
model = Model()
# model.load_state_dict(torch.load("../model_0.27.pth"))

In [3]:
print(model)

Model(
  (model): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu

In [4]:
# Load an example image
image_path = "../data/CityScapes/leftImg8bit/train/aachen/aachen_000001_000019_leftImg8bit.png"
image = Image.open(image_path)

# Show the image
image.show()

In [5]:
# Preprocess the image
image = preprocess(image)

# Make a prediction
with torch.no_grad():
    prediction = model(image)
    

In [6]:
print(prediction.shape)
print(prediction)

torch.Size([1, 20, 520, 1040])
tensor([[[[-0.0234, -0.0234, -0.0234,  ..., -0.0189, -0.0189, -0.0189],
          [-0.0234, -0.0234, -0.0234,  ..., -0.0189, -0.0189, -0.0189],
          [-0.0234, -0.0234, -0.0234,  ..., -0.0189, -0.0189, -0.0189],
          ...,
          [-0.0182, -0.0182, -0.0182,  ..., -0.0079, -0.0079, -0.0079],
          [-0.0182, -0.0182, -0.0182,  ..., -0.0079, -0.0079, -0.0079],
          [-0.0182, -0.0182, -0.0182,  ..., -0.0079, -0.0079, -0.0079]],

         [[-0.0330, -0.0330, -0.0330,  ..., -0.0413, -0.0413, -0.0413],
          [-0.0330, -0.0330, -0.0330,  ..., -0.0413, -0.0413, -0.0413],
          [-0.0330, -0.0330, -0.0330,  ..., -0.0413, -0.0413, -0.0413],
          ...,
          [-0.0428, -0.0428, -0.0428,  ..., -0.0434, -0.0434, -0.0434],
          [-0.0428, -0.0428, -0.0428,  ..., -0.0434, -0.0434, -0.0434],
          [-0.0428, -0.0428, -0.0428,  ..., -0.0434, -0.0434, -0.0434]],

         [[ 0.0091,  0.0091,  0.0091,  ...,  0.0006,  0.0006,  0.0006],

In [7]:
import torch.nn.functional as F
prediction_smax = F.softmax(prediction, dim=1)
print(prediction_smax.shape)
print(prediction_smax)

torch.Size([1, 20, 520, 1040])
tensor([[[[0.0494, 0.0494, 0.0494,  ..., 0.0497, 0.0497, 0.0497],
          [0.0494, 0.0494, 0.0494,  ..., 0.0497, 0.0497, 0.0497],
          [0.0494, 0.0494, 0.0494,  ..., 0.0497, 0.0497, 0.0497],
          ...,
          [0.0497, 0.0497, 0.0497,  ..., 0.0501, 0.0501, 0.0501],
          [0.0497, 0.0497, 0.0497,  ..., 0.0501, 0.0501, 0.0501],
          [0.0497, 0.0497, 0.0497,  ..., 0.0501, 0.0501, 0.0501]],

         [[0.0489, 0.0489, 0.0489,  ..., 0.0486, 0.0486, 0.0486],
          [0.0489, 0.0489, 0.0489,  ..., 0.0486, 0.0486, 0.0486],
          [0.0489, 0.0489, 0.0489,  ..., 0.0486, 0.0486, 0.0486],
          ...,
          [0.0485, 0.0485, 0.0485,  ..., 0.0484, 0.0484, 0.0484],
          [0.0485, 0.0485, 0.0485,  ..., 0.0484, 0.0484, 0.0484],
          [0.0485, 0.0485, 0.0485,  ..., 0.0484, 0.0484, 0.0484]],

         [[0.0510, 0.0510, 0.0510,  ..., 0.0506, 0.0506, 0.0506],
          [0.0510, 0.0510, 0.0510,  ..., 0.0506, 0.0506, 0.0506],
          [

In [8]:
prediction_argmax = torch.argmax(prediction_smax, dim=1)

print(prediction_argmax.shape)
print(prediction_argmax)
print(np.unique(prediction_argmax))

torch.Size([1, 520, 1040])
tensor([[[5, 5, 5,  ..., 5, 5, 5],
         [5, 5, 5,  ..., 5, 5, 5],
         [5, 5, 5,  ..., 5, 5, 5],
         ...,
         [5, 5, 5,  ..., 6, 6, 6],
         [5, 5, 5,  ..., 6, 6, 6],
         [5, 5, 5,  ..., 6, 6, 6]]])
[3 5 6]


In [9]:
# Postprocess the prediction
prediction = postprocess(prediction)

# Check size of prediction numpy array
print(prediction.shape)
print(np.unique(prediction))

(1, 520, 1040)
[3. 5. 6.]


In [10]:
# Turn the prediction into an image
prediction_s = prediction.squeeze(0)
prediction_image = Image.fromarray(prediction_s)
prediction_image.show()